In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install truecase
!pip install seqeval
!pip install transformers
!pip install datasets

In [ ]:
import os

os.chdir("drive/MyDrive/Colab Notebooks/kcl_individual_project")

In [ ]:
import torch
import data_preprocess as dp
import comp_models as cm
from transformers import AutoTokenizer
import transformers

# Hyperparameters: Note that if the "MAX_LEN" below is changed, the "MAX_LEN" in comp_models.py must be the same as well.
MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 5e-5

# load the tokenizer of roberta-base
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
# for conll2003
datasets = dp.load_conll("conll2003")

batch_size = 16
label_list1 = datasets["train"].features["pos_tags"].feature.names
label_list2 = datasets["train"].features["chunk_tags"].feature.names
label_list3 = datasets["train"].features["ner_tags"].feature.names
print(len(label_list1),len(label_list2),len(label_list3))

example = datasets["train"][4]
print(example["tokens"])
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
word_ids = tokenized_input.word_ids()
aligned_labels1 = [-100 if i is None else example["pos_tags"][i] for i in word_ids]
aligned_labels2 = [-100 if i is None else example["chunk_tags"][i] for i in word_ids]
aligned_labels3 = [-100 if i is None else example["ner_tags"][i] for i in word_ids]
print(aligned_labels1)
print(aligned_labels2)
print(aligned_labels3)

train_data_loader, val_data_loader, test_data_loader = dp.preprocess_raw_dataset(tokenizer, "conll2003", datasets, BATCH_SIZE, MAX_LEN)

In [ ]:
# for conll2000
datasets = dp.load_conll("conll2000")

batch_size = 16
label_list1 = datasets["train"].features["pos_tags"].feature.names
label_list2 = datasets["train"].features["chunk_tags"].feature.names
print(len(label_list1),len(label_list2))

example = datasets["train"][4]
print(example["tokens"])
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
word_ids = tokenized_input.word_ids()
aligned_labels1 = [-100 if i is None else example["pos_tags"][i] for i in word_ids]
aligned_labels2 = [-100 if i is None else example["chunk_tags"][i] for i in word_ids]
print(aligned_labels1)
print(aligned_labels2)

train_data_loader, test_data_loader = dp.preprocess_raw_dataset(tokenizer, "conll2000", datasets, BATCH_SIZE, MAX_LEN)
val_data_loader = []

In [ ]:
datasets

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
# the code for choosing which RoBERTa composite model to use
# basically, pick a model and its checkpoint_path in one of the following 3 groups: MRN, MTL or STL
"""
Group 1
for MRN
"""
model = cm.ROBERTA_MRN_conll2003()
# model = cm.ROBERTA_MRN_conll2000()
checkpoint_path = "roberta_reg_checkpoints/MRN_conll2003_15_07"


"""
Group 2
for MTL
"""
# model = cm.ROBERTA_MTL_conll2003()
# model = cm.ROBERTA_MTL_conll2000()
# checkpoint_path = "roberta_no_reg_checkpoints/MTL_conll2003_15_07"


"""
Group 3
for STL: need to specify the total number of distinct labels in the target task
for example, in the NER task in CONLL-03, num_labels is 9
"""
# num_labels = 23
# model = cm.ROBERTA_STL(num_labels)
# checkpoint_path = "roberta_base_checkpoints/STL_conll2003_0704_CHUNK"

# attach the model to the target device (normally GPU)
model.to(device)
optimizer = torch.optim.AdamW(params = model.parameters(), lr=LEARNING_RATE)

In [ ]:
model

In [ ]:
# load the model from a checkpoint for evaluation
# model, optimizer, EPOCHS , avg_valid_loss = cm.load_model(checkpoint_path, model, optimizer)
# model.to(device)

In [ ]:
# the code for training --> make sure the model is trained under the right learning paradigm with the right dataset
# for instance, train_conll2003_MRN() means that the model will be fine-tuned under MRN with CoNLL-03 dataset
# Note that train_STL() can be used for both of the datasets

import trainers as trainers
model = trainers.train_conll2003_MRN(EPOCHS, train_data_loader, val_data_loader, model, optimizer, checkpoint_path)
# model = trainers.train_conll2000_MRN(EPOCHS, train_data_loader, val_data_loader, model, optimizer, checkpoint_path)
# model = trainers.train_conll2003_MTL(EPOCHS, train_data_loader, val_data_loader, model, optimizer, checkpoint_path)
# model = trainers.train_conll2000_MTL(EPOCHS, train_data_loader, val_data_loader, model, optimizer, checkpoint_path)
# model = trainers.train_STL(EPOCHS, train_data_loader, val_data_loader, model, optimizer, checkpoint_path, num_labels, task="chunk")

In [ ]:
# the code for testing --> make sure the model is tested under the right learning paradigm with the right dataset
# for instance, test_conll2003_MRN_or_MTL() means that the MRN or MTL model will be tested with CoNLL-03 dataset
# Note that test_STL() can be used for both of the datasets

import testers as testers
testers.test_conll2003_MRN_or_MTL(model, test_data_loader, datasets)
# testers.test_conll2000_MRN_or_MTL(model, test_data_loader, datasets)
# testers.test_STL(model, test_data_loader, datasets, task="chunk")